# Assignment 1: 3D Reconstruction via Triangulation

## Submission Guide

Submit the following:

- A well-commented codebase demonstrating each step of the reconstruction process, with explanations of each implementation step, results of each step (calibrated K matrix, distortion coefficients, number of matched features, etc.), and visualizations of the 3D point cloud. You are allowed to use built-in functions from OpenCV, but you must check the official documentation for any function you use and include an explanation of how it works in your implementation.
- Create a zip folder with your last name and send the assignment to me via Teams messages.

### Submission Deadline: 05/02/2025 @ 23:59

---

## Objective

This assignment involves extracting common feature points from two images using the SIFT algorithm and reconstructing a 3D scene based on these points. You will learn camera calibration, feature matching, fundamental and essential matrix computation, and triangulation.

---



# Steps

## 1. Camera Calibration
**Theory**: Camera calibration involves determining the intrinsic ($K$) and extrinsic ($R, t$) parameters of the camera. The intrinsic matrix $K$ is defined as:

$$
K = \begin{bmatrix}
f_x & 0 & c_x \\
0 & f_y & c_y \\
0 & 0 & 1
\end{bmatrix}
$$

where $f_x$ and $f_y$ are the focal lengths, and $(c_x, c_y)$ is the optical center. Distortion coefficients account for lens imperfections.

**Mathematics**:
- Use the chessboard pattern to obtain 2D image points and corresponding 3D object points. Chessboard images are provided in the "cal" folder.
- The calibration process minimizes the reprojection error defined as:

$$
\text{Error} = \sum \| P_{\text{projected}} - P_{\text{actual}} \|^2
$$

**Task**:
- Capture a series of images of a checkerboard pattern.
- Detect corners and compute the intrinsic parameters using OpenCV.

```python

import cv2
import numpy as np

# Load calibration images
# Detect checkerboard corners
ret, corners = cv2.findChessboardCorners(gray, (cols, rows), None)

# Prepare object points and image points
# Perform calibration using cv2.calibrateCamera
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Compute reprojection error
reprojection_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    reprojection_error += error
reprojection_error /= len(objpoints)
print("Reprojection error:", reprojection_error)

```
---



## 2. Undistort Images
**Theory**: Lens distortion can cause straight lines to appear curved in images. The undistortion process corrects this, allowing for more accurate feature extraction.

**Task**:
- Use the intrinsic parameters to undistort your left (`left2`) and right (`right2`) images. These images are given in the "images" folder.

```python

# Undistort images using cv2.undistort

```
---



## 3. Feature Matching using SIFT
**Theory**: The SIFT (Scale-Invariant Feature Transform) algorithm extracts keypoints that are invariant to scale and rotation. Each keypoint has a descriptor that is robust to changes in lighting and perspective.

**Task**:
- Implement SIFT to extract keypoints and descriptors from the undistorted images.
- Use FLANN/Brute Force for feature matching and filter out good matches using Lowe’s ratio test:

$$
\frac{d_1}{d_2} < 0.75
$$

```python

# See the Feature Matching notebook I shared ;)

```
---

## 4. Compute the Fundamental Matrix
**Theory**: The fundamental matrix $F$ relates corresponding points in stereo images, satisfying the epipolar constraint:

$$
\mathbf{p_2}^T F \mathbf{p_1} = 0
$$

**Task**:
- Calculate the fundamental matrix using the matched points and apply RANSAC for robustness.

```python

F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)
```
---

## 5. Compute the Essential Matrix and Camera Pose
**Theory**: The essential matrix $E$ is derived from the fundamental matrix and intrinsic matrix:

$$
E = K^T F K
$$

**Task**:
- Calculate the essential matrix and recover the camera pose.

```python
E, _ = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC)
_, R, t, _ = cv2.recoverPose(E, pts1, pts2, K)
```
---

## 6. Triangulation
**Theory**: Triangulation computes the 3D position of points based on their 2D projections in two images. Given projection matrices $P_1$ and $P_2$:

$$
P = K[R|t]
$$

**Task**:
- Use the projection matrices to triangulate the 3D points.

```python
points_4D = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)
points_3D = points_4D[:3] / points_4D[3]
```
---


## 7. Bundle Adjustment (Optional)
**Theory**: Bundle adjustment refines the 3D structure and camera parameters simultaneously to minimize the reprojection error.

**Task**:
- Implement bundle adjustment using optimization libraries (e.g., SciPy).

---

## 8. Visualization
**Theory**: Visualizing the reconstructed 3D points provides insights into the geometry of the scene.

**Task**:
- Use Open3D to visualize the reconstructed 3D point cloud.

```python
import open3d as o3d

# Create and visualize the point cloud using Open3D
```
---

